# Extracting desired business reviews
In the last notebook we achieved two things. Firstly, we identified the business categories (Italian/Pizza) that our campaign was going to target. Secondly, we identified business IDs associated with those categories. Knowing the business IDs allows us to select just the reviews of interest from the reviews file. Before you go rushing into reading the review data into one big dataframe to then filter just those IDs of interest, as a data scientist you will have first got an idea of the size of the file. The review file is pretty big (nearly six million lines). Unless you have a very good computer with lots of RAM, or you particularly want to fire up your system/memory monitor and watch the free memory fall further and further until your computer seizes, you should be thinking in terms of how to read in only the lines of interest. This way, you only consume the minimum RAM necessary. Of course, in general, even then you should be making some sort of calculation as to whether even this would be within the capabilities of your hardware. In this case, it should be doable for most modernish computers. The task of this notebook is to parse the review data file and save the (much smaller) subset of interest.

In [165]:
# import the required libraries here
import pandas as pd
import matplotlib.pyplot as plt

## List of required business IDs
First we need to read in the list of business IDs we obtained previously.

In [172]:
# task: read in our previously created file that contains the business IDs of interest
# one line
businesses = pd.read_csv('output.csv')

In [173]:
businesses.shape

(9224, 3)

It's a good idea to have a quick check first.

In [174]:
businesses.head()

,business_id,review_count,stars
0,fweCYi8FmbJXHCqLnwuk8w,16,4.0
1,PZ-LZzSlhSe9utkQYU8pFg,40,4.0
2,BvYU3jvGd0TJ7IyZdfiN2Q,16,3.5
3,PkDghu4aan2_wxrhXjTEgg,16,3.5
4,RFbMVekR8lU9tPJ8sWrwHg,18,4.0


We want to grab just the business_id column as a list. It's worth also then double checking it matches the business_id column above.

In [111]:
business_ids = businesses['business_id'].values
print(business_ids[:5])

['fweCYi8FmbJXHCqLnwuk8w' 'PZ-LZzSlhSe9utkQYU8pFg'
 'BvYU3jvGd0TJ7IyZdfiN2Q' 'PkDghu4aan2_wxrhXjTEgg'
 'RFbMVekR8lU9tPJ8sWrwHg']


## Filtering a large data file in chunks
The review data file comprises some nearly 6 million lines. We don't want all of those lines. Our general approach will be to read through the file and check the business_id in each row. If this ID is contained in our list of wanted IDs, we will keep that row. It will be fastest, but most expensive for memory, to read the entire file from disk in one go and then filter the rows in memory. At the other extreme it will be most memory efficient to read in one record at a time, check whether we want it or not (and discard if not), but slowest. We will strike a balance where we choose a sufficiently large chunksize that still fits easily in memory but whereby we don't need to read in too many chunks.

In [133]:
# For convenience here, again we are accessing the data in the working directory that contains our notebooks.
# Normal best practise is to keep your data separate, but this keeps things simple here.
# If you're comfortable specifying a filepath to files outside of this working directory, then feel free to do so.
# task: create a reader object for the review json file
# Hint: use lines=True as before but add the chunksize=100000 parameter
# one line of code here
review_reader = pd.read_json('../../review.json', lines = True, chunksize = 100000)

We use the time magic for the next cell, out of interest, to time how long it takes. If you have the time and are curious, you're welcome to play around with the chunksize and monitor your system memory use and the time this cell takes, but _caveat emptor_! I suggest you save your notebook before running this cell, just in case.

In [134]:

# task: process the file one chunk at a time,
# filter that chunk for rows with a business_id in business_ids
# You can either do this within in a loop, having initialized an empty list,
# or using a more pythonic list comprehension

#by using %%time variable is unable to initialize
reviews = [frames.loc[frames.business_id.apply(lambda x: x in business_ids)] for frames in review_reader]

# (this took some 24 minutes on my old i7)

In [131]:
%time
y = 1
y

1

All being well, you now have the desired reviews read in. But we don't yet have them in a convenient DataFrame:

In [132]:
#it will not identify y because in the last cell it is not exposed as we are using cell magic function(time)
print(y)

1


In [135]:
type(reviews)

list

In [147]:
len(reviews)

67

In [159]:
list1 = [[1,2],[4,5],[6,7],[8,9]]
df1 = pd.DataFrame(list1, columns = ['A','B'])
print(df1)
df2 = pd.DataFrame(list1, columns = ['A','B'])
df = [df1,df2]
list2 = [4,6,7]
list3=[df3.A.apply(lambda x: x) for df3 in df ]
print(list3)
print(len(list3))


   A  B
0  1  2
1  4  5
2  6  7
3  8  9
[0    1
1    4
2    6
3    8
Name: A, dtype: int64, 0    1
1    4
2    6
3    8
Name: A, dtype: int64]
2


In [142]:
reviews

[                  business_id  cool                date  funny  \
 5      eU_713ec6fTGNO4BegRaww     0 2013-01-20 13:25:59      0   
 12     FxLfqxdYPA6Z85PFKaqLrg     0 2016-05-07 01:36:53      0   
 15     YvrylyuWgbP90RgMqZQVnQ     0 2017-04-07 21:27:49      0   
 19     y-Iw6dZflNix4BdwIyTNGA     0 2014-06-27 21:19:23      0   
 20     qx6WhZ42eDKmBchZDax4dQ     0 2017-03-27 01:14:37      0   
 22     I4Nr-MVc26qWr08-S3Q1ow     0 2016-01-17 05:26:22      0   
 27     Gyrez6K8f1AyR7dzW9fvAw     1 2013-12-28 22:28:08      0   
 34     YSUcHqlKMPHHJ_cTrqtNrA     0 2017-12-29 13:55:19      0   
 35     N0apJkxIem2E8irTBRKnHw     0 2015-01-18 15:30:50      0   
 36     sMzNLdhJZGzYirIWt-fMAg     0 2015-06-21 00:59:14      0   
 53     eoyvbnRYQe-z85e8Rc6vAg     0 2013-08-15 15:00:14      0   
 67     aNe8ofTYrealxqv7VtFTuw     0 2015-07-16 07:31:28      2   
 69     ZNB91myFoOYgyXoG5LQeGQ     0 2014-12-17 19:34:47      0   
 73     74C1z6cqEdvJVgETTcEodw     0 2014-12-17 17:46:03      

In [150]:
reviews_copy = reviews


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
5,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0,fdiNeiN_hoCxCMy2wTRW9g,4,I'll be the first to admit that I was not exci...,0,w31MKYsNFMrjhWxxAb5wIw
12,FxLfqxdYPA6Z85PFKaqLrg,0,2016-05-07 01:36:53,0,Z7wgXp98wYB57QdRY3HQ3w,4,Wow. So surprised at the one and two star revi...,0,GYNnVehQeXjty0xH7-6Fhw
15,YvrylyuWgbP90RgMqZQVnQ,0,2017-04-07 21:27:49,0,svK3nBU7Rk8VfGorlrN52A,5,You can't really find anything wrong with this...,0,NJlxGtouq06hhC7sS2ECYw
19,y-Iw6dZflNix4BdwIyTNGA,0,2014-06-27 21:19:23,0,4bUyL7lzoWzDZaJETAKREg,3,Good selection of classes of beers and mains. ...,0,_N7Ndn29bpll_961oPeEfw
20,qx6WhZ42eDKmBchZDax4dQ,0,2017-03-27 01:14:37,0,Amo5gZBvCuPc_tZNpHwtsA,5,"Our family LOVES the food here. Quick, friendl...",1,DzZ7piLBF-WsJxqosfJgtA
22,I4Nr-MVc26qWr08-S3Q1ow,0,2016-01-17 05:26:22,0,IByf6mVY0WA838XNSh-5MA,4,The food is always good and the prices are rea...,0,3CJUJILq7CLHk_9OrvpvQg
27,Gyrez6K8f1AyR7dzW9fvAw,1,2013-12-28 22:28:08,0,qm97yMwREr7BKkexlwLFbg,5,their pettuccine was fresh-made in the morning...,0,TTRVdTXKcq-xn6-1IWbwEw
34,YSUcHqlKMPHHJ_cTrqtNrA,0,2017-12-29 13:55:19,0,E6B-2U2sGG3xgmnNWZAEew,1,Came here on a Thursday night at 6:30 p.m. My ...,0,DbccYu3OppWKl21OanZnTg
35,N0apJkxIem2E8irTBRKnHw,0,2015-01-18 15:30:50,0,wOv7JiO0EGcJ61P2JcDS1g,2,Went here last weekend and was pretty disappoi...,0,en0xQXlvRk-ZtKlaW4I8eQ
36,sMzNLdhJZGzYirIWt-fMAg,0,2015-06-21 00:59:14,0,RLbWoIri29BcQ8yjzsvQLQ,5,This place epitomizes the rumored transformati...,0,_o740mSNRhMNYuPjSJoPLg


Convert your reviews into a DataFrame now. _Hint: you may find pandas concat method useful here._

In [160]:
# task convert your reviews into a DataFrame
# one line of code here
reviews = pd.concat(reviews)

In [161]:
reviews.shape

(594862, 9)

In [162]:
reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
5,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0,fdiNeiN_hoCxCMy2wTRW9g,4,I'll be the first to admit that I was not exci...,0,w31MKYsNFMrjhWxxAb5wIw
12,FxLfqxdYPA6Z85PFKaqLrg,0,2016-05-07 01:36:53,0,Z7wgXp98wYB57QdRY3HQ3w,4,Wow. So surprised at the one and two star revi...,0,GYNnVehQeXjty0xH7-6Fhw
15,YvrylyuWgbP90RgMqZQVnQ,0,2017-04-07 21:27:49,0,svK3nBU7Rk8VfGorlrN52A,5,You can't really find anything wrong with this...,0,NJlxGtouq06hhC7sS2ECYw
19,y-Iw6dZflNix4BdwIyTNGA,0,2014-06-27 21:19:23,0,4bUyL7lzoWzDZaJETAKREg,3,Good selection of classes of beers and mains. ...,0,_N7Ndn29bpll_961oPeEfw
20,qx6WhZ42eDKmBchZDax4dQ,0,2017-03-27 01:14:37,0,Amo5gZBvCuPc_tZNpHwtsA,5,"Our family LOVES the food here. Quick, friendl...",1,DzZ7piLBF-WsJxqosfJgtA


## Save the output
Having done all this hard work filtering our review file, and documenting the process in this notebook, we need to save the output. Go ahead and save the reviews DataFrame to a csv called 'reviews_filtered.csv'.

In [ ]:
# task: save the DataFrame to the specified file now.
# don't forget to use index=False
reviews.to_csv("reviews_filtered.csv", index = False)

If you inspect the file listing for the original json file and our new csv file, you should see we've gone from around 4.4 GB down to 325 MB. This is much more manageable!

# Summary
Great work! So far in our project you've seen how to break a problem down and identify relevant data, use data to gain insight and make a decision, and then leverage that knowledge to extract a desired subset of data from an otherwise unmanageable larger file. In the next notebook we'll dive into this data.